# Milestone 3 - Great Expectation

---

# i. Introduction

Name : Ryan Dionenatan

Batch : FTDS-038-RMT

# ii. Import Libraries

The first thing that I'll do is to import `great_expectations` as the tool to validate expectations on the data, and `shutil` to delete previoudly created folder.

In [1]:
# Import Libraries

from great_expectations.data_context import FileDataContext
import shutil

# iii. Data Loading

In this part, I begin with deleting `great` folder if there was one in order to not interfere with the new data context that will be made after that, then I'll create a new data context as a place for Great Expectation to store its result.

In [2]:
# Delete 'great' folder if there was one (this code is added because it's a one off check)
shutil.rmtree('great/', ignore_errors=True, onerror=None)

# Create a data context

context = FileDataContext.create(project_root_dir='./great/')

Next, I create a new datasource name and load the clean CSV file which I'm going to validate.

In [3]:
# Give a name to a Datasource

datasource_name = 'dealer-data-1'
datasource = context.sources.add_pandas(datasource_name)

# Give a name to a data asset
asset_name = 'dealer-data'
path_to_data = '/mnt/d/Hacktiv8/RMT-038/Phase 2/p2-ftds038-rmt-m3-orange-birds/P2M3_ryan_dionenatan_data_clean.csv'
asset = datasource.add_csv_asset(asset_name, filepath_or_buffer=path_to_data)

# Build batch request
batch_request = asset.build_batch_request()

After that, I create a new expectation suite so I can start to validate the clean CSV file.

In [4]:
# Create an expectation suite
expectation_suite_name = 'expectation-dataset'
context.add_or_update_expectation_suite(expectation_suite_name)

# Create a validator using above expectation suite
validator = context.get_validator(
    batch_request = batch_request,
    expectation_suite_name = expectation_suite_name
)

# Check the validator
validator.head()

Calculating Metrics:   0%|          | 0/1 [00:00<?, ?it/s]

,car_id,date,customer_name,gender,annual_income,dealer_name,company,model,engine,transmission,color,price,dealer_no,body_style,phone,dealer_region
0,C_CND_000001,2022-01-02,Geraldine,Male,13500,Buddy Storbeck's Diesel Service Inc,Ford,Expedition,Double Overhead Camshaft,Auto,Black,26000,06457-3834,SUV,8264678,Middletown
1,C_CND_000002,2022-01-02,Gia,Male,1480000,C & M Motors Inc,Dodge,Durango,Double Overhead Camshaft,Auto,Black,19000,60504-7114,SUV,6848189,Aurora
2,C_CND_000003,2022-01-02,Gianna,Male,1035000,Capitol KIA,Cadillac,Eldorado,Overhead Camshaft,Manual,Red,31500,38701-8047,Passenger,7298798,Greenville
3,C_CND_000004,2022-01-02,Giselle,Male,13500,Chrysler of Tri-Cities,Toyota,Celica,Overhead Camshaft,Manual,Pale White,14000,99301-3882,SUV,6257557,Pasco
4,C_CND_000005,2022-01-02,Grace,Male,1465000,Chrysler Plymouth,Acura,TL,Double Overhead Camshaft,Auto,Red,24500,53546-9427,Hatchback,7081483,Janesville


# iv. Expectation Check

## iv.1. Column `car_id` must be unique

The first thing that I'm going to check is if `car_id` column is unique since ID should never have the same value.

In [5]:
# Expectation 1 : Column `car_id` must be unique

validator.expect_column_values_to_be_unique('car_id')

Calculating Metrics:   0%|          | 0/8 [00:00<?, ?it/s]

{
  "success": true,
  "result": {
    "element_count": 23906,
    "unexpected_count": 0,
    "unexpected_percent": 0.0,
    "partial_unexpected_list": [],
    "missing_count": 0,
    "missing_percent": 0.0,
    "unexpected_percent_total": 0.0,
    "unexpected_percent_nonmissing": 0.0
  },
  "meta": {},
  "exception_info": {
    "raised_exception": false,
    "exception_traceback": null,
    "exception_message": null
  }
}

The result above shows that `car_id` column is indeed unique.

## iv.2. Column `annual_income` must be in between $10,000 to $15,000,000

The next thing that I'm going to check is if `annual_income` column is in between $10,000 to $15,000,000.

In [6]:
# Expectation 2 : Column `annual_income` must be in between $10,000 to $15,000,000

validator.expect_column_values_to_be_between(
    column='annual_income', min_value=10000, max_value=15000000
)

Calculating Metrics:   0%|          | 0/8 [00:00<?, ?it/s]

{
  "success": true,
  "result": {
    "element_count": 23906,
    "unexpected_count": 0,
    "unexpected_percent": 0.0,
    "partial_unexpected_list": [],
    "missing_count": 0,
    "missing_percent": 0.0,
    "unexpected_percent_total": 0.0,
    "unexpected_percent_nonmissing": 0.0
  },
  "meta": {},
  "exception_info": {
    "raised_exception": false,
    "exception_traceback": null,
    "exception_message": null
  }
}

The results above shows that the annual income data in the dataset satisfies the expectation.

## iv.3. Column `transmission` must contain either `Auto` or `Manual`

After that, I'm going to check if `transmission` column data is either `Auto` or `Manual`.

In [7]:
# Expectation 3 : Column `transmission` must contain either `Auto` or `Manual`

validator.expect_column_values_to_be_in_set('transmission', ['Auto', 'Manual'])

Calculating Metrics:   0%|          | 0/8 [00:00<?, ?it/s]

{
  "success": true,
  "result": {
    "element_count": 23906,
    "unexpected_count": 0,
    "unexpected_percent": 0.0,
    "partial_unexpected_list": [],
    "missing_count": 0,
    "missing_percent": 0.0,
    "unexpected_percent_total": 0.0,
    "unexpected_percent_nonmissing": 0.0
  },
  "meta": {},
  "exception_info": {
    "raised_exception": false,
    "exception_traceback": null,
    "exception_message": null
  }
}

The results above shows that the data in `transmission` column also satisfies the expectation.

## iv.4. Column `price` must have `int64` data type

Next, I'm going to check if `price` column actually have `int64` data type.

In [8]:
# Expectation 4 : Column `price` must have `int64`` data type

validator.expect_column_values_to_be_in_type_list('price', ['int64'])

Calculating Metrics:   0%|          | 0/1 [00:00<?, ?it/s]

{
  "success": true,
  "result": {
    "observed_value": "int64"
  },
  "meta": {},
  "exception_info": {
    "raised_exception": false,
    "exception_traceback": null,
    "exception_message": null
  }
}

The results above shows that the data type in `price` column satisfies the expectation.

## iv.5. Column `gender` character to be between 4-6

After that, I'm going to check if the data on `gender` column character to be between 4-6, in this case Male or Female.

In [9]:
# Expectation 5 : Column `gender` character to be between 4-6

validator.expect_column_value_lengths_to_be_between('gender', min_value=4, max_value=6)

Calculating Metrics:   0%|          | 0/9 [00:00<?, ?it/s]

{
  "success": true,
  "result": {
    "element_count": 23906,
    "unexpected_count": 0,
    "unexpected_percent": 0.0,
    "partial_unexpected_list": [],
    "missing_count": 0,
    "missing_percent": 0.0,
    "unexpected_percent_total": 0.0,
    "unexpected_percent_nonmissing": 0.0
  },
  "meta": {},
  "exception_info": {
    "raised_exception": false,
    "exception_traceback": null,
    "exception_message": null
  }
}

The results above shows that the character length in `gender` column satisfies the expectation.

## iv.6. Column `dealer_no` to have `-` in the value

Next, I'm going to check if the dealer registration number to have `-` between the numbers.

In [10]:
# Expectation 6 : Column `dealer_no` to have `-` in the value

validator.expect_column_values_to_match_regex('dealer_no', regex=".*-.*")

Calculating Metrics:   0%|          | 0/8 [00:00<?, ?it/s]

{
  "success": true,
  "result": {
    "element_count": 23906,
    "unexpected_count": 0,
    "unexpected_percent": 0.0,
    "partial_unexpected_list": [],
    "missing_count": 0,
    "missing_percent": 0.0,
    "unexpected_percent_total": 0.0,
    "unexpected_percent_nonmissing": 0.0
  },
  "meta": {},
  "exception_info": {
    "raised_exception": false,
    "exception_traceback": null,
    "exception_message": null
  }
}

The results above shows that the data in `dealer_no` column satisfies the expectation.

## iv.7. Total columns in the dataset to be exactly 16

The last thing that I'm going to validate is if the column in the dataset to be exactly 16.

In [11]:
# Expectation 7 : Total columns in the dataset to be exactly 16

validator.expect_table_column_count_to_equal(value=16)

Calculating Metrics:   0%|          | 0/3 [00:00<?, ?it/s]

{
  "success": true,
  "result": {
    "observed_value": 16
  },
  "meta": {},
  "exception_info": {
    "raised_exception": false,
    "exception_traceback": null,
    "exception_message": null
  }
}

The results above shows that column count expectation to be exactly 16 is satisfied.

## iv.8. Saving Expectation Suite

Once I know that all of the expectations are satisfied, I'm going to save the expectation suite so the new data that comes from the database can be checked in the same fashion as before.

In [12]:
# Save as an Expectation Suite

validator.save_expectation_suite()

Thank you.